Nom, prénom, groupe

# TP7 : Reconnaissance d'une diagonale dans carré 2x2

## Chargement des librairies

In [ ]:
# Bibliothèques utilisées

from matplotlib import pyplot as plt # graphique
import numpy as np # mathématiques
import pandas as pd # manipulation des données

print('Bibliothèques chargées')

## Chargement des données

In [ ]:
# Lecture des données depuis un fichier
data=pd.read_csv("diag2x2.csv",sep=",")
print(data)

# Problème A (à traiter avec l'enseignant)

Les données $(X_j,Y_j)$, $j\in\{0,\ldots,15\}$, ci-dessus représentent les 16 carrés composés de 4 cases 
<table><tr><td >x1</td><td>x2</td></tr><tr><td>x3</td><td>x4</td></tr></table>
chaque case pouvant prendre la valeur 0 ou 1, ainsi que la valeur $Y_j=0$ ou $Y_j=1$ selon que le carré comporte *exactement* une diagonale de 1 (les deux autres cases étant à 0).

Dans la suite, on tente de déterminer une fonction définie par l'expression $$R_{W,W_1,W_2}(x_1,x_2,x_3,x_4)=\sigma\bigg(h_W\bigg(\sigma\big(h_{W_1}(x_1,x_2,x_3,x_4)\big),\sigma(h_{W_2}\big(x_1,x_2,x_3,x_4\big)\bigg)\bigg)$$ avec
$$W=(b,w_1,w_2)\text{ et }h_W(s_1,s_2)=b+w_1s_1+w_2s_2$$
$$W_1=(b_1,w_{11},w_{12},w_{13},w_{14})\text{ et }h_{W_1}(x_1,x_2,x_3,x_4)=b_1+w_{11}x_1+w_{12}x_2+w_{13}x_3+w_{14}x_4,$$
$$W_2=(b_2,w_{21},w_{22},w_{23},w_{24})\text{ et }h_{W_2}(x_1,x_2,x_3,x_4)=b_2+w_{21}x_1+w_{22}x_2+w_{23}x_3+w_{24}x_4,$$
permettant d'approximer au mieux $Y_j$ à travers $R(x_{j1},x_{j2},x_{j3},x_{j4})$, pour chaque donnée $j$.

**Exercice**

Les 16 carrés sont enregistrés en Python sous forme de ligne de 5 valeurs, numérotées de 0 à 15.
Quels sont les numéros des données correspondant à un carré comportant exactement une diagonale de 1 ?

Réponse : lignes n° ... et n° ...

## Création des données

In [ ]:
# Extraction des données du tableau de données ("dataframe") chargé :
# extraction des entrées Xj
Xdata=data.drop(['Y'],axis=1) # la variable Xdata contient toutes les colonnes sauf la colonne Y (de l'axe 1="colonnes")
# extraction des sorties Yj
Ydata=data['Y'] # La variable Ydata contient la colonne Y

n=len(Xdata) # Nombre de lignes de données (ici 16)
d=len(Xdata.iloc[0]) # Nombre de colonnes de X[0] (ici 4)
print("Données et tailles des données définies")

## Modélisation par un réseau à 1 couche cachée de 2 neurones


**Exercice**

Représenter (sur papier) par un graphe de calcul le réseau de neurones associé à la fonction $X\mapsto R(X)$, où $X=(x_1,x_2,x_3,x_4)$.

**Exercice**

Compléter la cellule ci-dessous afin de définir la fonction `R(W,W1,W2,j)`, qui doit retourner la valeur de $R(X_j)$ avec comme paramètres entrant :
- `W`, qui représente la liste `[b,w_1,w_2]` (paramètres associés à $h_W$)
- `W1` et `W2`, qui représentent les listes `[b1,w11,w12,w13,w14]` (paramètres associés à $h_{W_1}$) et `[b2,w21,w22,w23,w24]` (paramètres associés à $h_{W_2}$) respectivement
- `j`, qui représente le numéro $j$ de la donnée considérée

In [ ]:
# Réseau
def sigma(x):
    return 1/(1+np.exp(-x))

def R(W,W1,W2,j):
    Xj=Xdata.iloc[j] # Xj=[Xj[0],Xj[1],Xj[2],Xj[3]] est la ligne j du tableau Xdata
    hW1=...
    hW2=...
    hW=...
    R=...
    return R

print('Fonction sigma et R définies')

**Exercice**

Compléter la cellule suivante afin de vérifier que la fonction $$R_{W,W_1,W_2}(X)=\sigma(-25+28\sigma(h_{W_1}(X))-13\sigma(h_{W_2}(X)))$$ avec $$h_{W_1}(X)=0-3x_1+1.5x_2+1.5x_3+9x_4$$
et $$h_{W_2}(X)=-14-4x_1+4x_2+4x_3+14x_4$$
répond au problème posé.

In [ ]:
W1=...
W2=...
W=...
for j in range(n):
    print('Y observé :',Ydata.iloc[j],' Y calculé par la fonction R:',R(W,W1,W2,j))

## Apprentissage du réseau de neurones

La détermination des coeffcients `W=[b,w1,w2]`, `W1=[b1,w11,w12,w13,w14]` et `W2=[b2,w21,w22,w23,w24]` permettant d'approximer la relation entre $X$ et $Y$ par $Y=R_{W,W_1,W_2}(X)$ se fait, à partir des données, par minimisation de la fonction définie par $$f(W,W_1,W_2)=\frac{1}{2}\sum\limits_{j=0}^{15}(R_{W,W_1,W_2}(X)-Y_j)^2.$$

La minimisation se fait par descente de gradient.

Puisque la dérivée d'une somme est la somme des dérivées, le gradient de $f$ est égal à la somme des gradients des fonctions $E_j$, définies pour chaque $j$ par $$E_j(W,W_1,W_2)=\frac{1}{2}(R_{W,W_1,W_2}(X)-Y_j)^2.$$

La fonction python `gradE` ci-dessous permet de calculer *par retropropagation* le gradient de $E_j$ par rapport à *chacun des paramètres* contenu dans les listes $W$, $W_1$ et $W_2$.

In [ ]:
# Gradient par rétropropagation

def gradE(W,W1,W2,j):
    b,w1,w2=W[0],W[1],W[2]
    b1,w11,w12,w13,w14=W1[0],W1[1],W1[2],W1[3],W1[4]
    b2,w21,w22,w23,w24=W2[0],W2[1],W2[2],W2[3],W2[4]
    Xj=Xdata.iloc[j]
    Yj=Ydata.iloc[j]
    h1=b1+w11*Xj[0]+w12*Xj[1]+w13*Xj[2]+w14*Xj[3]
    h2=b2+w21*Xj[0]+w22*Xj[1]+w23*Xj[2]+w24*Xj[3]    
    H=b+w1*sigma(h1)+w2*sigma(h2)
    sH=sigma(H)
    sh1=sigma(h1)
    sh2=sigma(h2)
    dsH=sH*(1-sH)
    dsh1=sh1*(1-sh1)
    dsh2=sh2*(1-sh2)
    dEdsH=sH-Yj
    dsHdb=dsH*1
    dsHdw1=dsH*sh1
    dsHdw2=dsH*sh2
    dsHdsh1=dsH*w1
    dsHdsh2=dsH*w2
    dsh1db1=dsh1*1
    dsh1dw11=dsh1*Xj[0]
    dsh1dw12=dsh1*Xj[1]
    dsh1dw13=dsh1*Xj[2]
    dsh1dw14=dsh1*Xj[3]
    dsh2db2=dsh2*1
    dsh2dw21=dsh2*Xj[0]
    dsh2dw22=dsh2*Xj[1]
    dsh2dw23=dsh2*Xj[2]
    dsh2dw24=dsh2*Xj[3]
    dEdb=dEdsH*dsHdb
    dEdw1=dEdsH*dsHdw1
    dEdw2=dEdsH*dsHdw2
    dEdb1=dEdsH*dsHdsh1*dsh1db1
    dEdw11=dEdsH*dsHdsh1*dsh1dw11
    dEdw12=dEdsH*dsHdsh1*dsh1dw12
    dEdw13=dEdsH*dsHdsh1*dsh1dw13
    dEdw14=dEdsH*dsHdsh1*dsh1dw14
    dEdb2=dEdsH*dsHdsh2*dsh2db2
    dEdw21=dEdsH*dsHdsh2*dsh2dw21
    dEdw22=dEdsH*dsHdsh2*dsh2dw22
    dEdw23=dEdsH*dsHdsh2*dsh2dw23
    dEdw24=dEdsH*dsHdsh2*dsh2dw24
    return [[dEdb,dEdw1,dEdw2],[dEdb1,dEdw11,dEdw12,dEdw13,dEdw14],[dEdb2,dEdw21,dEdw22,dEdw23,dEdw24]]

**Exercice**

À l'aide du graphe de calcul de la fonction $R$ (réseau de neurones), expliciter chacune des dérivées partielles qu'il est nécessaire de calculer pour obtenir la valeur de $\frac{\partial E_5}{\partial w_{23}}(W,W_1,W_2)$, et vérifier ce calcul dans ce code de la fonction `gradE` ci-dessus.

*Réponse:*

Dérivées partielles utilisées pour calculer $\frac{\partial E_5}{\partial w_{23}}(W,W_1,W_2)$ :
- Dérivée de ... par rapport à ..., identifié par la variable ... dans le code
- Dérivée de ... par rapport à ..., identifié par la variable ... dans le code
- ...

**Exercice**

Utiliser la cellule ci-dessous pour calculer $R_{W,W_1,W_2}(X_5)$ et $\frac{\partial E_5}{\partial w_{23}}(W,W_1,W_2)$ en :
$$W=(b,w_1,w_2)=(1,0,0)$$
$$W_1=(b_1,w_{11},w_{12},w_{13},w_{14})=(1,1,1,1,1)$$
et $$W_2=(b_1,w_{21},w_{22},w_{23},w_{24})=(1,0,0,0,0).$$

In [ ]:
W=...
W1=...
W2=...
gradE5=...
print('$R_{W,W_1,W_2}(X_5)=$',...)
print('$\frac{\partial E_5}{\partial w_{23}}(W,W_1,W_2)=$',...)

## Algorithme de descente de gradient

La fonction suivante implémente une *version partielle* de l'algorithme de descente de gradient.

Comme il y a 16 données, la fonction f est une somme de 16 carrés.

L'algorithme de descente gradient, visant à minimiser $f$, doit rendre chacun des carrés petits (sans augmenter les autres).

L'algorithme de descente de gradient ci-dessous effectue une descente de gradient sur seulement 5 termes de la somme, qui sont *choisis au hasard* à chaque pas de l'algorithme.

Cette méthode (standard) diminue les calculs effectués à chaque pas de l'algorithme, tout en permettant en général de *probablement* réduire la valeur de $f$.

In [ ]:
def descente(W,W1,W2,tau=0.01,tolerance=1e-2,NbIterationsMax=1000):
    diverge=False
    for i in range(NbIterationsMax):
        batch=np.random.randint(0, 16, 5) # 5 entiers aléatoires choisis entre 0 et 15
        g=[[0,0,0],[0,0,0,0,0],[0,0,0,0,0]] # gradient courant, initialisé à un gradient nul
        for j in batch: # j parcourt 5 données aléatoires
            gg=gradE(W,W1,W2,j) # calcul du gradient de la fonction associée à la donnée n°j
            g = [np.add(g[0],gg[0]),np.add(g[1],gg[1]),np.add(g[2],gg[2])] # cumul du gradient gg au gradient courant            
        try: # traitement des erreurs si l'algorithme diverge
            if (W[0]==float('inf')) or (W[1]==float("inf")) or (W[2]==float("inf"))\
            or (W1[0]==float('inf')) or (W1[1]==float("inf")) or (W1[2]==float("inf")) or (W1[3]==float("inf")) or (W1[4]==float("inf"))\
            or (W2[0]==float('inf')) or (W2[1]==float("inf")) or (W2[2]==float("inf")) or (W2[3]==float("inf")) or (W2[4]==float("inf")):
                raise(OverflowError)
            ng=np.sqrt(np.sum([np.sum([gi**2 for gi in g[k]]) for k in range(2)]))/len(batch) # somme des carrés des coordonnées de g
            if ng<tolerance:
                print('L\'algorithme a convergé en',i,'itérations. \nSolution atteinte :\n W=',W,'\n W1=',W1,'\n W2=',W2,'\nGradient :',g)
                return [W,W1,W2]
            W=[W[k]-tau*g[0][k] for k in range(3)]
            W1=[W1[k]-tau*g[1][k] for k in range(5)]
            W2=[W2[k]-tau*g[2][k] for k in range(5)]
        except OverflowError as err: # traitement de l'erreur "overflow"
            print('L\'algorithme a divergé \nSolution atteinte :\n W=',W,'\n W1=',W1,'\n W2=',W2,'\nGradient :',g)
            diverge=True
            break
    if (diverge==False):
        print('L\'algorithme n\'a pas convergé \nSolution atteinte :\n W=',W,'\n W1=',W1,'\n W2=',W2,'\nGradient :',g,'\n Norme :',ng)
    return [W,W1,W2]

print('Fonction descente définie')

**Exercice**

Dans l'algorithme de descente de gradient ci-dessus, que désigne la variable `batch` et que calcule les lignes de code suivantes :

`g=[[0,0,0],[0,0,0,0,0],[0,0,0,0,0]]`  
`for j in batch:`  
`    gg=gradE(W,W1,W2,j)`  
`    g = [np.add(g[0],gg[0]),np.add(g[1],gg[1]),np.add(g[2],gg[2])]`
?
    
*Réponse :*

...

**Exercice**

En utilisant la cellule suivante, faire converger l'algorithme avec une tolérance de $10^{-4}$.

In [ ]:
descente([1,0,0],[1,1,1,1,1],[1,0,0,0,0],0.01,0.0001,10000)

**Exercice**

Compléter la boucle suivante afin de vérifier que les paramètres atteints par l'algorithme de descente répondent au problème posé.

In [ ]:
W=...
W1=...
W2=...
for j in range(n):
    print('Y observé :',Ydata.iloc[j],' Y calculé :',R(W,W1,W2,j))

# Problème B (à faire en autonomie)

Déterminer les coefficients d'un réseau de neurones qui identifie si un carré de 4 cases contient exactement une ligne de 1.

À cette fin, vous pourrez au préalable modifier les données `Ydata[j]`, en modifiant les valeurs correspondant aux carrés visés :
- Ydata[j] doit être égal à 1 si le carré contient exactement une ligne de 1
- Ydata[j] doit être égal à 0 si le carré ne contient pas exactement une ligne de 1

(par exemple `Ydata[10]=0` affecte la valeur $0$ à $Y_{10}$).


In [ ]:
# Modification des données Ydata :
Ydata[...]=...
Ydata[...]=...
Ydata[...]=...
Ydata[...]=...

# Vérification des données :

for j in range(n):
    print('X :',Xdata.iloc[j],', Y correspondant :',Ydata.iloc[j])

In [ ]:
# Faire convergeer l'algorithme (tolerance à 10^-4) pour ces nouvelles données :

descente([1,0,0],[1,1,1,1,1],[1,0,0,0,0],0.01,0.0001,10000)

In [ ]:
# Vérification des valeurs de la fonction R obtenue :

W=...
W1=...
W2=...
for j in range(n):
    print('Y observé :',Ydata.iloc[j],' Y calculé :',R(W,W1,W2,j))